In [1]:
import numpy as np
from PIL import Image
import math

# ---------------------------------------
# LOAD YOUR IMAGE (Grayscale Conversion)
# ---------------------------------------
img = Image.open("C:/Users/Jayaram/Pictures/GRAYSCALE.jpeg").convert("L")
I = np.asarray(img).astype(np.float32)

# Normalize to [0,1]
I_norm = I / 255.0
epsilon = 1e-6

# ---------------------------------------
# LOG DOMAIN
# ---------------------------------------
log_I = np.log(I_norm + epsilon)

# ---------------------------------------
# MANUAL BOX FILTER (NO BUILT-IN FUNCTIONS)
# ---------------------------------------
def manual_box_filter(image, kernel_size):
    h, w = image.shape
    k = kernel_size
    pad = k // 2

    # Pad the image
    padded = np.zeros((h + 2*pad, w + 2*pad), dtype=np.float32)
    padded[pad:pad+h, pad:pad+w] = image

    output = np.zeros_like(image)
    area = k * k

    # Naive nested loop
    for y in range(h):
        for x in range(w):
            window = padded[y:y+k, x:x+k]
            output[y, x] = np.sum(window) / area

    return output

# ---------------------------------------
# ESTIMATE ILLUMINATION (LOW FREQUENCY)
# ---------------------------------------
kernel_size = 41  # You can try 31, 51 also
log_L_hat = manual_box_filter(log_I, kernel_size)

# ---------------------------------------
# ESTIMATE REFLECTANCE (HIGH FREQUENCY)
# ---------------------------------------
log_R_hat = log_I - log_L_hat

# Convert back
R_hat = np.exp(log_R_hat)

# Normalize to [0,1]
R_hat = (R_hat - np.min(R_hat)) / (np.max(R_hat) + 1e-6)

# Convert to 8-bit
R_hat_8bit = (R_hat * 255).astype(np.uint8)

# ---------------------------------------
# SAVE RESULT
# ---------------------------------------
Image.fromarray(R_hat_8bit).save("reflectance_grayscale_output.jpg")

print("Grayscale reflectance image saved as reflectance_grayscale_output.jpg")


Grayscale reflectance image saved as reflectance_grayscale_output.jpg


In [3]:
import numpy as np
from PIL import Image

# ---------------------------------------------------
# LOAD YOUR COLOR IMAGE
# ---------------------------------------------------
img_color = Image.open("C:/Users/Jayaram/Pictures/GRAYSCALE.jpeg").convert("RGB")
I_color = np.asarray(img_color).astype(np.float32)

# Normalize
R = I_color[:, :, 0] / 255.0
G = I_color[:, :, 1] / 255.0
B = I_color[:, :, 2] / 255.0

epsilon = 1e-6

# ---------------------------------------------------
# LOG DOMAIN PER CHANNEL
# ---------------------------------------------------
log_R = np.log(R + epsilon)
log_G = np.log(G + epsilon)
log_B = np.log(B + epsilon)

# ---------------------------------------------------
# AVERAGE LOG-INTENSITY FOR JOINT ILLUMINATION
# ---------------------------------------------------
log_intensity = (log_R + log_G + log_B) / 3.0


# ---------------------------------------------------
# MANUAL BOX FILTER (REUSING SAME FUNCTION)
# ---------------------------------------------------
def manual_box_filter(image, kernel_size):
    h, w = image.shape
    k = kernel_size
    pad = k // 2

    padded = np.zeros((h + 2*pad, w + 2*pad), dtype=np.float32)
    padded[pad:pad+h, pad:pad+w] = image

    output = np.zeros_like(image)
    area = float(k*k)

    for y in range(h):
        for x in range(w):
            window = padded[y:y+k, x:x+k]
            output[y, x] = np.sum(window) / area

    return output

# Large kernel = smooth illumination
kernel_size = 41
log_L_hat = manual_box_filter(log_intensity, kernel_size)

# ---------------------------------------------------
# REFLECTANCE PER CHANNEL
# ---------------------------------------------------
log_Rr = log_R - log_L_hat
log_Rg = log_G - log_L_hat
log_Rb = log_B - log_L_hat

# Convert back & normalize
def normalize_channel(c):
    c = np.exp(c)
    c = c - np.min(c)
    c = c / (np.max(c) + 1e-6)
    return c

Rr = normalize_channel(log_Rr)
Rg = normalize_channel(log_Rg)
Rb = normalize_channel(log_Rb)

# Convert back to image
Rr8 = (Rr * 255).astype(np.uint8)
Rg8 = (Rg * 255).astype(np.uint8)
Rb8 = (Rb * 255).astype(np.uint8)

output_color = np.stack([Rr8, Rg8, Rb8], axis=2)
result_img = Image.fromarray(output_color)

result_img.save("reflectance_color_output.jpg")

print("Color reflectance saved as reflectance_color_output.jpg")


Color reflectance saved as reflectance_color_output.jpg
